# Alpha 전략 1

- momentum
- reversion
- normalized rdv/adv (volume peak)을 가미한 momentum & reversion 

## Basic settings

### Import libraries

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path

from sklearn.impute import SimpleImputer

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
## custom library

import eda_util as eutil
import submission_config as subconfig
import submission_util as subutil

In [3]:
pd.set_option('display.float_format', lambda x: f'{x:,g}')

In [4]:
BASE_PATH = subconfig.BASE_PATH
DATA_PATH = subconfig.DATA_PATH

OUTPUT_PATH = subconfig.OUTPUT_PATH

### Import data & preprocessing

In [5]:
krx_df = pd.read_csv(subconfig.krx_df_PATH)

In [6]:
krx_df.columns = ['date', 'code', 'name', 'volume', 'open', 'high', 'low', 'close']

In [7]:
krx_df['date'] = pd.to_datetime(krx_df['date'], format='%Y%m%d')

In [8]:
return_df = pd.read_pickle(subconfig.return_df_PATH)
close_df = pd.read_pickle(subconfig.adjclose_df_PATH)

In [9]:
## date list

holidays = return_df.isnull().all(axis=1)
tradingdays = ~holidays

holidays = holidays.index[holidays]
tradingdays = tradingdays.index[tradingdays]

In [21]:
TRAIN_START = pd.to_datetime(subconfig.TRAIN_START, format='%Y-%m-%d')
SIMOS_END = pd.to_datetime(subconfig.SIMOS_END, format='%Y-%m-%d')

In [22]:
tradingdays = tradingdays[(tradingdays >= TRAIN_START) & (tradingdays <= SIMOS_END)]

In [12]:
dacon_sid_list = [ii[1:] for ii in krx_df['code'].unique()]

In [13]:
return_df = return_df.loc[tradingdays, :].dropna(axis='columns', how='all')
return_df = return_df.loc[:, dacon_sid_list]

close_df = close_df.loc[tradingdays, :].dropna(axis='columns', how='all')
close_df = close_df.loc[:, dacon_sid_list]

In [23]:
SIMOS_START = subconfig.SIMOS_START
# simOS_END = subconfig.SIMOS_END

### Import additional data

In [15]:
volume_df = pd.read_pickle(subconfig.volume_df_PATH)
dollarvolume_df = pd.read_pickle(subconfig.dollarvolume_df_PATH)
marketcap_df = pd.read_pickle(subconfig.marketcap_df_PATH)
market_cat_df = pd.read_pickle(DATA_PATH / 'market_cat_df_20140101_20230705.pickle')

In [16]:
volume_df = volume_df.loc[tradingdays, :].dropna(axis='columns', how='all')
volume_df = volume_df.loc[:, dacon_sid_list]

dollarvolume_df = dollarvolume_df.loc[tradingdays, :].dropna(axis='columns', how='all')
dollarvolume_df = dollarvolume_df.loc[:, dacon_sid_list]

marketcap_df = marketcap_df.loc[tradingdays, :].dropna(axis='columns', how='all')
marketcap_df = marketcap_df.loc[:, dacon_sid_list]

In [17]:
volume_df = volume_df.shift(1)
dollarvolume_df = dollarvolume_df.shift(1)
marketcap_df = marketcap_df.shift(1)

In [18]:
market_cat_inrange = market_cat_df[market_cat_df['trdDd'].isin(tradingdays)]

In [19]:
KOSPI_sid_list = market_cat_inrange[market_cat_inrange['is_KOSPI'] == True]['ISU_SRT_CD'].unique()
KOSDAQ_sid_list = market_cat_inrange[market_cat_inrange['is_KOSDAQ'] == True]['ISU_SRT_CD'].unique()
KONEX_sid_list = market_cat_inrange[market_cat_inrange['is_KONEX'] == True]['ISU_SRT_CD'].unique()

### Parameters

In [20]:
PORTFOLIO_DATE = subconfig.PORTFOLIO_DATE

RDVADV_WINDOW = subconfig.WINDOWS['rdvadv'] # 20

### normalized RDV/ADV signal

In [24]:
adv_df = dollarvolume_df.rolling(RDVADV_WINDOW, ).mean().dropna(axis='rows', how='all')

분모: average RDV/ADV ratio


In [25]:
avg_adv_s = adv_df.mean(axis='columns')
avg_rdv_s = dollarvolume_df.iloc[RDVADV_WINDOW:, :].mean(axis='columns')

In [26]:
avg_rdvadv_s = avg_rdv_s / avg_adv_s

분자: individual RDV/ADV ratio 

In [27]:
ii_rdvadv_df = dollarvolume_df.iloc[RDVADV_WINDOW:, :] / adv_df

In [28]:
normalized_rdvadv_signal_df = ii_rdvadv_df.divide(avg_rdvadv_s, axis='rows')
normalized_rdvadv_signal_df

ISU_SRT_CD,060310,095570,006840,054620,265520,211270,027410,282330,126600,138930,...,243070,084110,145020,024060,010240,189980,000540,003280,037440,238490
trdDd,,,,,,,,,,,,,,,,,,,,,
2021-06-29,0.455129,0.580815,0.886328,1.78965,0.741888,1.73313,0.613687,0.606445,1.21465,0.998485,...,1.10244,2.93848,0.515822,0.868417,2.37328,0.374486,1.14816,NaN,0.226014,0.919051
2021-06-30,0.415442,0.240279,0.405709,0.726369,0.475821,0.907576,0.4201,1.10324,0.539159,0.755196,...,1.81974,0.798739,0.631614,0.340136,14.3229,0.996211,0.600184,NaN,0.369902,0.630198
2021-07-01,0.314977,0.212304,0.465795,0.931287,0.470661,0.514831,0.648498,1.99712,0.354948,0.501368,...,4.39418,8.5227,0.745765,0.319291,7.08257,0.416926,1.40616,NaN,0.291685,0.618186
2021-07-02,0.399208,0.295054,0.570299,0.366269,0.496024,0.584884,1.26369,3.66506,0.356092,0.701445,...,1.03469,1.22015,0.426687,0.971435,1.91106,0.786424,0.705502,NaN,0.495063,0.955606
2021-07-05,1.02063,0.146543,0.639072,0.410245,0.842014,0.260692,1.97162,1.5627,0.724157,0.519005,...,2.03734,0.881352,0.244086,2.64395,0.377858,0.422299,0.421016,NaN,1.13061,0.666445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-15,1.62098,0.32888,1.97189,0.104643,0.95799,0.361154,0.726808,0.519196,0.606503,0.525978,...,0.2784,0.643094,0.586189,0.394554,0.739696,0.649907,0.679339,1.1706,0.662143,0.144598
2023-06-16,0.292674,0.588688,1.12985,0.124622,0.90052,0.31337,0.973597,1.10088,0.442653,1.00447,...,0.592776,0.71876,0.580354,0.487591,0.570073,0.493214,1.08468,0.789117,0.514062,0.106241
2023-06-19,0.12223,3.09939,1.38265,0.322443,0.710468,0.296359,1.33772,2.43478,0.495513,1.70201,...,0.44873,0.936876,0.771501,0.2652,2.44716,1.00167,0.93063,0.793416,0.350493,0.14148


In [29]:
MODEL_TRAIN_START = pd.to_datetime('2021-06-29', format='%Y-%m-%d')

## Alphas

### 1. momentum

모멘텀에는 다양한 종류가 있다. 

여러 기술적 분석 지표를 이용하여 momentum을 구현해보도록 하겠다. 

#### 단순 mean return 이용

n day reversion 하듯 그냥 단순히 지난 3일간의 return을 mean하여 그대로 betting

In [31]:
return_df.rolling(3).mean() 

ISU_SRT_CD,060310,095570,006840,054620,265520,211270,027410,282330,126600,138930,...,243070,084110,145020,024060,010240,189980,000540,003280,037440,238490
trdDd,,,,,,,,,,,,,,,,,,,,,
2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-03,0.0011771,-0.0113272,0.0090764,-0.000893152,0.00286173,0.00489254,0.00331375,0.0126996,5.21407e-05,-0.000828594,...,-0.00844797,-0.0186622,0.00846125,0.0179485,0.0106986,0.00952562,0.0117334,0,0.0888121,4.44127e-05
2021-06-04,0.00346402,-0.0121856,0.0148195,-0.0157569,0.000595172,-0.0157625,0.000948958,-0.00161768,0.00216519,1.63252e-05,...,-0.00467534,-0.00238534,0.0281736,0.00491915,0.00695755,-0.00321887,0.00613309,0,0.0436129,0.00725942
2021-06-07,0.028346,-0.0223156,-0.0152584,0.00948125,-0.00112501,0.000271284,0.000944945,0.0120584,-0.00351869,0.000850702,...,0.00645955,0.0111836,0.0306884,-0.00202825,-0.00194047,-0.0175652,-0.00387472,0,-0.0187325,0.010158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-15,-0.00540223,0.0026567,0.0119185,-0.00792373,0.00706487,-0.00765828,-0.00284011,0.00642651,-0.0149801,-0.00384724,...,-0.0105253,0.00169138,-0.0115263,-0.00294881,-0.0107692,0.00450784,-0.0137333,-0.0245452,-0.010333,-0.0199753
2023-06-16,-0.00877072,0.0219032,0.02354,-0.0108032,5.52192e-05,-0.00294718,-0.00489996,0.0145168,-0.00728782,-0.000446527,...,-0.00678353,0.00333333,-0.00665645,-0.00117153,0.0106188,0.0077841,-0.00821116,-0.0192504,-0.00584942,-0.000243099
2023-06-19,0.00800263,0.017203,0.0110754,-0.00940259,5.52192e-05,0.0144069,-0.00944918,-0.0051443,0.00752868,0.00340704,...,-0.00415205,0.0115356,-0.00257244,-0.00235566,0.0156933,0.00948587,-0.00416591,0.00111712,0.00975107,0.00585983


#### 캔들차트 적삼병 이용

적삼병: 3 거래일 연속 시가 < 종가 로 마무리되는 빨간 캔들이 3개 찍히는 것

### 2. reversion

### 3. news + momentum

단순 모멘텀을 넘어, news를 찾고, 그 news 이후의 모멘텀을 노려본다. 

### 4. volume peak + reversion